In [1]:
import os
os.environ['ALFRED_ROOT'] = '/root/home/hoyeung/alfred/'

import sys
sys.path.append(os.path.join(os.environ['ALFRED_ROOT']))
sys.path.append(os.path.join(os.environ['ALFRED_ROOT'], 'models'))

# from argparse import ArgumentDefaultsHelpFormatter, ArgumentParser
import torch
import pprint
import json
import numpy as np
from data.preprocess import Dataset
from importlib import import_module, reload
from argparse import ArgumentDefaultsHelpFormatter, ArgumentParser
from collections import Counter
# from models.utils.helper_utils import optimizer_to

In [2]:
parser = ArgumentParser(formatter_class=ArgumentDefaultsHelpFormatter)
args = parser.parse_args('')

# settings
args.data = '/root/data_alfred/unlabeled_12k_20201206/seen'
args.pp_folder = 'pp'
args.pframe = 300
args.fast_epoch = True

In [3]:
vocab = torch.load(os.path.join('/root/home/hoyeung/blob_alfred_data/json_feat_2.1.0_backup_20200826_agent_training', "%s.vocab" % args.pp_folder))

In [4]:
vocab

{'word': Vocab(2360), 'action_low': Vocab(15), 'action_high': Vocab(93)}

## Preprocess original train splits oct21

In [5]:
# SKIP preprocessing. Already preprocessed

## Preprocess augmented splits

In [13]:
args.splits = '/root/data_alfred/splits/unlabeled_12k_20201206.json'
args.toy_splits = '/root/data_alfred/splits/debug_20201210.json'

with open(args.splits, 'r') as f:
    splits = json.load(f)

print([(k,len(splits[k])) for k in splits.keys()])

[('augmentation', 11947)]


In [14]:
splits['augmentation'][0]

{'task': 'pick_and_place_simple-CellPhone-None-Sofa-224/trial_T20190912_062415_255725',
 'repeat_idx': 0,
 'full_traj_success': True,
 'collected_subgoals': 5}

In [15]:
args.fast_epoch = False
dataset = Dataset(args, vocab)
dataset.preprocess_splits_augmentation(splits, lm_tag='baseline')

100% (11947 of 11947) |##################| Elapsed Time: 0:05:05 Time:  0:05:05


In [16]:
args.fast_epoch = False
dataset = Dataset(args, vocab)
dataset.preprocess_splits_augmentation(splits, lm_tag='explainer')

100% (11947 of 11947) |##################| Elapsed Time: 0:04:57 Time:  0:04:57


In [17]:
args.fast_epoch = False
dataset = Dataset(args, vocab)
dataset.preprocess_splits_augmentation(splits, lm_tag='explainer_full')

100% (11947 of 11947) |##################| Elapsed Time: 0:05:03 Time:  0:05:03


# Make SPLITS

## Make Toy Training + Augmentation Split

In [18]:
alfred_splits_p = '/root/data_alfred/splits/oct21.json'
with open(alfred_splits_p, 'r') as f:
    alfred_splits = json.load(f)
print([(k,len(alfred_splits[k])) for k in alfred_splits.keys()])

[('tests_seen', 1533), ('tests_unseen', 1529), ('train', 21023), ('valid_seen', 820), ('valid_unseen', 821)]


In [19]:
toy_aug_splits_p = '/root/data_alfred/splits/debug_20201210.json'
with open(toy_aug_splits_p, 'r') as f:
    toy_aug_splits = json.load(f)

print([(k,len(toy_aug_splits[k])) for k in toy_aug_splits.keys()])

[('augmentation', 198)]


In [20]:
toy_agent_tr_splits = {}

for k in alfred_splits.keys():
    toy_agent_tr_splits[k] = alfred_splits[k][:16]

toy_agent_tr_splits['augmentation'] = toy_aug_splits['augmentation']

print([(k,len(toy_agent_tr_splits[k])) for k in toy_agent_tr_splits.keys()])

[('tests_seen', 16), ('tests_unseen', 16), ('train', 16), ('valid_seen', 16), ('valid_unseen', 16), ('augmentation', 198)]


In [21]:
toy_split_p = '/root/data_alfred/splits/agent_augmentation_20201230_toy.json'

with open(toy_split_p, 'w') as f:
    json.dump(toy_agent_tr_splits, f)

In [22]:
with open(toy_split_p, 'r') as f:
    dum = json.load(f)

In [23]:
dum

{'tests_seen': [{'repeat_idx': 0, 'task': 'trial_T20190909_042500_949430'},
  {'repeat_idx': 1, 'task': 'trial_T20190909_042500_949430'},
  {'repeat_idx': 2, 'task': 'trial_T20190909_042500_949430'},
  {'repeat_idx': 0, 'task': 'trial_T20190908_064522_624362'},
  {'repeat_idx': 1, 'task': 'trial_T20190908_064522_624362'},
  {'repeat_idx': 2, 'task': 'trial_T20190908_064522_624362'},
  {'repeat_idx': 0, 'task': 'trial_T20190909_085207_886285'},
  {'repeat_idx': 1, 'task': 'trial_T20190909_085207_886285'},
  {'repeat_idx': 2, 'task': 'trial_T20190909_085207_886285'},
  {'repeat_idx': 0, 'task': 'trial_T20190908_132517_128346'},
  {'repeat_idx': 1, 'task': 'trial_T20190908_132517_128346'},
  {'repeat_idx': 2, 'task': 'trial_T20190908_132517_128346'},
  {'repeat_idx': 0, 'task': 'trial_T20190908_133706_349398'},
  {'repeat_idx': 1, 'task': 'trial_T20190908_133706_349398'},
  {'repeat_idx': 2, 'task': 'trial_T20190908_133706_349398'},
  {'repeat_idx': 0, 'task': 'trial_T20190911_082512_6786

# Make SPLITS

## Read in existing splits

In [25]:
original_alfred_splits_p = '/root/data_alfred/splits/oct21.json'

with open(original_alfred_splits_p, 'r') as f:
    original_alfred_splits = json.load(f)
    
print([(k,len(original_alfred_splits[k])) for k in original_alfred_splits.keys()])

[('tests_seen', 1533), ('tests_unseen', 1529), ('train', 21023), ('valid_seen', 820), ('valid_unseen', 821)]


In [27]:
augmented_splits_p = '/root/data_alfred/splits/unlabeled_12k_20201206.json'

with open(augmented_splits_p, 'r') as f:
    augmented_splits = json.load(f)

print([(k,len(augmented_splits[k])) for k in augmented_splits.keys()])

[('augmentation', 11947)]


## 1. Annotate all failures with Explainer and with Baseline.  Retrain agent and evaluate

In [28]:
experiment_1_splits = {}

for k in original_alfred_splits.keys():
    experiment_1_splits[k] = original_alfred_splits[k] 

experiment_1_splits['augmentation'] = augmented_splits['augmentation']

print([(k,len(experiment_1_splits[k])) for k in experiment_1_splits.keys()])

[('tests_seen', 1533), ('tests_unseen', 1529), ('train', 21023), ('valid_seen', 820), ('valid_unseen', 821), ('augmentation', 11947)]


In [29]:
experiment_1_split_path = '/root/data_alfred/splits/data_augmentation_experiment1_20201230.json'
with open(experiment_1_split_path, 'w') as f:
    json.dump(experiment_1_splits, f)
    
with open(experiment_1_split_path, 'r') as f:
    dum = json.load(f)

print([(k,len(dum[k])) for k in dum.keys()])

[('tests_seen', 1533), ('tests_unseen', 1529), ('train', 21023), ('valid_seen', 820), ('valid_unseen', 821), ('augmentation', 11947)]


In [ ]:
agent_augmentation_20201230_toy.json